# YOLOv8 Training for PPE and Chair Detection

This notebook facilitates training the refined YOLOv8 model on Google Colab, leveraging the data stored in your Google Drive.

In [ ]:
import os

dataset_path = "/kaggle/input/d"

print("Files in dataset:")
print(os.listdir(dataset_path))

# Explore the structure
print("\nDataset structure:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    print(f"\n{item}:")
    if os.path.isdir(item_path):
        print(f"  Type: Folder")
        try:
            contents = os.listdir(item_path)
            print(f"  Contains {len(contents)} items")
            print(f"  First few: {contents[:5]}")
        except Exception as e:
            print(f"  Error: {e}")
    else:
        print(f"  Type: File")

Files in dataset:
['spiky1991']

Dataset structure:

spiky1991:
  Type: Folder
  Contains 1 items
  First few: ['ai-cam']


In [10]:
# 3. Install dependencies
!pip install ultralytics

In [11]:
import os

dataset_path = "/kaggle/input/d"

# Check if the expected folders exist
paths_to_check = [
    f"{dataset_path}/merged-dataset/train/images",
    f"{dataset_path}/merged-dataset/valid/images",
    f"{dataset_path}/merged-dataset/test/images"
]

print("Checking dataset structure:")
for path in paths_to_check:
    exists = os.path.exists(path)
    print(f"{'✓' if exists else '✗'} {path}")
    if exists:
        num_files = len(os.listdir(path))
        print(f"  Contains {num_files} files")

Checking dataset structure:
✗ /kaggle/input/d/merged-dataset/train/images
✗ /kaggle/input/d/merged-dataset/valid/images
✗ /kaggle/input/d/merged-dataset/test/images


In [12]:
import os

dataset_path = "/kaggle/input/d"

# Show what's actually in the dataset
print("What's inside /kaggle/input/d:")
print(os.listdir(dataset_path))

# Explore deeper
print("\n" + "="*50)
print("Full structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    
    # Show directories
    subindent = ' ' * 2 * (level + 1)
    for dir_name in dirs:
        print(f'{subindent}📁 {dir_name}/')
    
    # Show first few files
    for file in files[:3]:
        print(f'{subindent}📄 {file}')
    if len(files) > 3:
        print(f'{subindent}... and {len(files)-3} more files')
    
    # Only go 3 levels deep to avoid too much output
    if level > 2:
        break

What's inside /kaggle/input/d:
['spiky1991']

Full structure:
d/
  📁 spiky1991/
  spiky1991/
    📁 ai-cam/
    ai-cam/
      📁 runs/
      📁 constructiondataset/
      📁 assets/
      📁 output/
      📁 source_files/
      📁 chair-dataset/
      📁 results/
      📁 output_detections/
      📁 models/
      📁 merged-dataset/
      📁 data/
      📄 webcam_inference.py
      📄 merge_datasets.py
      📄 .gitignore
      ... and 4 more files
      runs/
        📁 detect/


In [13]:
import os

# Correct base path
dataset_path = "/kaggle/input/d/spiky1991/ai-cam"
working_path = "/kaggle/working"

# First, let's verify the merged-dataset structure
merged_path = f"{dataset_path}/merged-dataset"
print("Contents of merged-dataset:")
print(os.listdir(merged_path))

# Check if train/valid/test folders exist
for split in ['train', 'valid', 'test']:
    split_path = os.path.join(merged_path, split)
    if os.path.exists(split_path):
        print(f"\n✓ {split}/")
        contents = os.listdir(split_path)
        print(f"  Contains: {contents}")
        
        # Check images folder
        images_path = os.path.join(split_path, 'images')
        if os.path.exists(images_path):
            num_images = len(os.listdir(images_path))
            print(f"  images/: {num_images} files")

Contents of merged-dataset:
['valid', 'test', 'train']

✓ train/
  Contains: ['labels', 'labels.cache', 'images']
  images/: 2723 files

✓ valid/
  Contains: ['labels', 'labels.cache', 'images']
  images/: 144 files

✓ test/
  Contains: ['labels', 'images']
  images/: 82 files


In [14]:
import os

# Correct paths for Kaggle
dataset_path = "/kaggle/input/d/spiky1991/ai-cam"
working_path = "/kaggle/working"

# Create YAML file
yaml_path = os.path.join(working_path, 'merged_data.yaml')

new_content = f"""
train: {dataset_path}/merged-dataset/train/images
val: {dataset_path}/merged-dataset/valid/images
test: {dataset_path}/merged-dataset/test/images

nc: 11
names: ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle', 'Chair']
"""

with open(yaml_path, 'w') as f:
    f.write(new_content)
    
print(f"✓ Created {yaml_path}")
print(f"\nYAML content:\n{new_content}")

# Verify the paths exist
print("\nVerifying paths:")
for split in ['train', 'valid', 'test']:
    path = f"{dataset_path}/merged-dataset/{split}/images"
    exists = os.path.exists(path)
    if exists:
        count = len(os.listdir(path))
        print(f"✓ {split}: {count} images")
    else:
        print(f"✗ {split}: NOT FOUND")

✓ Created /kaggle/working/merged_data.yaml

YAML content:

train: /kaggle/input/d/spiky1991/ai-cam/merged-dataset/train/images
val: /kaggle/input/d/spiky1991/ai-cam/merged-dataset/valid/images
test: /kaggle/input/d/spiky1991/ai-cam/merged-dataset/test/images

nc: 11
names: ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle', 'Chair']


Verifying paths:
✓ train: 2723 images
✓ valid: 144 images
✓ test: 82 images


In [15]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model from Ultralytics
model = YOLO('yolov8n.pt')  # Will auto-download if not present
# Or use: yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt

print("Starting training on merged dataset...")

# Train the model
results = model.train(
    data='/kaggle/working/merged_data.yaml',
    epochs=50,
    imgsz=640,
    batch=64,
    device=0,  # Change to 0 if GPU available
    project='/kaggle/working/runs',
    name='ppe_detection',
    patience=10
)

print("Training completed!")
print(f"Best model saved at: /kaggle/working/runs/ppe_detection/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Starting training on merged dataset...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/merged_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7